In [1]:
%load_ext autoreload
%autoreload 2
import os
import pandas as pd
import cv2
import sys
import random
sys.path.append('../')

random.seed(42)

In [2]:
from faces_clustering import Clusterer, FeatureExtractor, is_image, get_files_folder

Using TensorFlow backend.


# Extracting Features

In [ ]:
complete_urls = get_files_folder("/home/paulo/Documentos/slnp_faces_leg_55/", is_image)

In [ ]:
df_features_senet50 = FeatureExtractor('senet50').extract(complete_urls)
df_features_senet50.head(20)

In [ ]:
df_features_senet50.to_pickle('../data/leg55_senet50.pkl')

In [ ]:
df_features_resnet50 = FeatureExtractor('resnet50').extract(complete_urls)
df_features_resnet50.head(20)

In [ ]:
df_features_resnet50.to_pickle('../data/leg55_resnet50.pkl')

In [ ]:
df_features_vgg16 = FeatureExtractor('vgg16').extract(complete_urls)
df_features_vgg16.head(20)

In [ ]:
df_features_vgg16.to_pickle('../data/leg55_vgg16.pkl')

# Clustering

In [8]:
df_embeddings = pd.read_pickle('../data/leg55_senet50.pkl')

In [10]:
classes = df_embeddings.urls.apply(lambda x: x.split('/')[-2])

In [11]:
valid_indexes = df_embeddings.embeddings.apply(lambda x: str(x) != '-')
df_embeddings = df_embeddings.loc[valid_indexes]

In [12]:
df_embeddings['classes'] = classes

/home/paulo/.local/share/virtualenvs/face_clustering-EKR70IVd/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


unique_classes = list(df_embeddings.classes.unique())

samples_classes = random.sample(unique_classes, 5)

sample_indices =  df_embeddings.classes.apply(lambda x: x in samples_classes)

In [17]:
clusterer = Clusterer(n_clusters=514, face_embeddings = df_embeddings,algs=['kmeans', 'affinity', 'agglomerative'])

In [18]:
clusters, models_inst = clusterer.clusterize()

In [20]:
clusters

,urls,embeddings,classes,cluster_kmeans,cluster_affinity,cluster_agglomerative
0,/home/paulo/Documentos/slnp_faces_leg_55/74385...,"[0.17480865, 0.24839097, 0.018389545, 0.398058...",74385_b,2,598,262
1,/home/paulo/Documentos/slnp_faces_leg_55/74385...,"[0.07091311, 0.04615592, 0.0, 6.4110007, 3.386...",74385_b,424,0,269
4,/home/paulo/Documentos/slnp_faces_leg_55/74385...,"[0.10499452, 2.5098183, 0.0, 5.429726, 0.61141...",74385_b,424,0,269
5,/home/paulo/Documentos/slnp_faces_leg_55/74385...,"[0.017898438, 1.6189704, 0.0022666005, 5.99468...",74385_b,424,0,269
6,/home/paulo/Documentos/slnp_faces_leg_55/74385...,"[0.02720209, 4.4259987, 0.006993143, 5.375417,...",74385_b,424,0,269
...,...,...,...,...,...,...
9133,/home/paulo/Documentos/slnp_faces_leg_55/67138...,"[0.046600726, 12.429699, 0.37234017, 0.0286798...",67138_b,256,681,100
9134,/home/paulo/Documentos/slnp_faces_leg_55/67138...,"[0.07046378, 15.637359, 0.12425924, 0.06128547...",67138_b,256,681,100
9135,/home/paulo/Documentos/slnp_faces_leg_55/67138...,"[0.045246698, 5.049192, 0.02735675, 0.01505866...",67138_b,256,681,100
9136,/home/paulo/Documentos/slnp_faces_leg_55/67138...,"[0.04391961, 9.75497, 0.14378008, 1.3651364, 5...",67138_b,256,681,100


In [19]:
clusters.to_pickle('../data/leg55_clusters.pkl')

In [ ]:
clusters.sort_values(by=['urls']).head()

In [ ]:
models_inst['cluster_kmeans'].cluster_centers_

In [17]:
clusters.groupby(['cluster_kmeans']).count().describe()

,urls,embeddings,classes
count,514.000000,514.000000,514.000000
mean,16.805447,16.805447,16.805447
std,15.284217,15.284217,15.284217
min,1.000000,1.000000,1.000000
25%,11.000000,11.000000,11.000000
50%,16.000000,16.000000,16.000000
75%,20.000000,20.000000,20.000000
max,330.000000,330.000000,330.000000


# Evaluating

In [28]:
from sklearn import metrics

In [29]:
metrics.homogeneity_score(clusters.classes,clusters.cluster_agglomerative)

0.9541468089117898

In [30]:
metrics.completeness_score(clusters.classes, clusters.cluster_agglomerative)

0.9620517846793569

In [31]:
metrics.v_measure_score(clusters.classes, clusters.cluster_agglomerative)

0.9580829914287667